In [1]:
# -*- coding: utf-8 -*-
import urllib
import pandas as pd
import selenium
import scrapy
import requests
import wget
import sys
import os
from scrapy.http import TextResponse
if not sys.modules.get('selenium'):
    os.system('pip3 install selenium')
from selenium import webdriver
if not sys.modules.get('requests'):
    os.system('pip3 install requests')
    import requests
if not sys.modules.get('bs4'):
    os.system('pip install bs4')
    from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
if not sys.modules.get('numpy'):
    os.system("pip install numpy")
    import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
import re 
if not sys.modules.get('pyautogui'):
    os.system('pip install pyautogui')
    import pyautogui
from selenium.webdriver.common.by import By
import ssl

In [2]:
def clean_text(text):
    import re
    text = re.sub(r'[\n\t\r]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub('<.+?>', ' ', text)
    cleaned = text.strip()
    return(cleaned)

In [38]:
global site
site = pd.read_excel("C:/Users/park/Desktop/사이트크롤링/site_park.xlsx",sheet_name="Sheet1",header=2)
site = site.loc[site.부처 != "한국산업기술평가관리원",:]
site = site.loc[site.부처 != "정보통신기획평가원_사업공고",:]
site = site.loc[site.부처 != "정보통신산업진흥원",:]
site = site.loc[site.부처 != "한국엔지니어링협회",:]
site = site.reset_index(drop=True)
urls = list(site["MainUrl1"])

In [22]:
global driver
driver = webdriver.Chrome("C:/Users/park/Desktop/사이트크롤링/chromedriver.exe")
driver.implicitly_wait(3)

In [5]:
#경로를 만들고 만들어진 경로로 저장위치를 변경한다.
def createdir(directory):
    try:
        
        #경로 생성
        if os.makedirs(directory):
            #경로위치 변경
            os.chdir(directory)
        else:
            os.chdir(directory)
    except OSError as e:
        print("Failed to create directory!!!!!")

In [39]:
def part_save2(number):
    global site,driver
    try:
        title = driver.find_element_by_css_selector(site["title"][number]).text
    except:
        iframe(number)
        title = driver.find_element_by_css_selector(site["title"][number]).text
    try:
        date = driver.find_element_by_css_selector(site["date"][number]).text
    except:
        date = "-"
    try:
        img_exist1 = driver.find_elements_by_tag_name(site["imgs_css1"][number])
    except:
        img_exist1 = "-"
    try:
        img_exist2 = driver.find_elements_by_tag_name(site["img_css2"][number])
    except:
        img_exist2 = "-"
    try:
        img_exist3 = driver.find_elements_by_tag_name(site["img_css3"][number])
    except:
        img_exist3 = "-"
    try:
        contents_exist = driver.find_elements_by_tag_name(site["contents"][number])
    except:
        contents_exist_sub = driver.find_elements_by_tag_name(site["contents_sub"][number])
    contents = []
    if img_exist1 != "-":
        for l in range(0,len(img_exist1)):
            contents.append(img_exist1[l].get_attribute('src'))
        if contents_exist:
            for l in range(0,len(contents_exist)):
                contents.append(contents_exist[l].text)
    elif img_exist2 != "-":
        for l in range(0,len(img_exist2)):
            contents.append(img_exist2[l].get_attribute('src'))
    elif img_exist3 != "-":
        for l in range(0,len(img_exist3)):
            contents.append(img_exist3[l].get_attribute('src'))
    elif contents_exist:
        for l in range(0,len(contents_exist)):
            contents.append(contents_exist[l].text)
    elif contents_exist_sub != "-":
        for l in range(0,len(contents_exist_sub)):
            contents.append(contents_exist[l].text)
    contents_1 = "-"
    try:
        while '' in contents: contents.remove('')
    except:
        pass
    try:
        while ' ' in contents: contents.remove(' ')
    except:
        pass
    download_exist = driver.find_elements_by_tag_name(site["download_exist"][number])
    if download_exist:
        donwload_name = []
        download = []
        for k in range(0,len(download_exist)):
            download.append(download_exist[k].get_attribute('href'))
            donwload_name.append(download_exist[k].text)
    else:
        download = "-"
        donwload_name = "-"
    return title,date,contents,download,donwload_name,contents_1

In [42]:
def save2(number):
    global part_save2,site,driver
    part = part_save2(number)
    title = part[0]
    date = part[1]
    contents = part[2]
    site_dir = 'C:/Users/park/Desktop/sample/%s'%site["부처"][number]
    title_dir = site_dir + "/" + title
    createdir(title_dir)
    pattern = re.compile('(png|jpg)')
    for n in range(0,len(contents)):
        if pattern.search(contents[n]):
            try:
                ssl._create_default_https_context = ssl._create_unverified_context
                urllib.request.urlretrieve(contents[n],'output%s.jpg'%n)
            except:
                wget.download(contents[n],'output%s.jpg'%n,verify=False)
            contents[n] = "-"
        else:
            contents = part[2]
    download = len(part[3])
    if part[3] == "-":
        download = "0"
    elif download > 0:
        #파일 다운로드
        try:
            for i in range(0,len(part[3])):
                wget.download(part[3][i],"%s"%part[4][i])
        except:
            download = part[4]
    os.chdir(site_dir)
    dic = pd.Series({'제목' : title,'올린날' : date,'내용' : contents,'첨부파일갯수' : download})
    dataframe  =  pd.DataFrame(dic).T
    return dataframe

In [41]:
def A_site2(number):
    global site,driver
    data_frame = pd.DataFrame(columns = ["제목","올린날","내용","첨부파일갯수"])
    driver.get(urls[number])
    time.sleep(2)
    list_num = range(int(site["range_row"][number]),int(site["range_high"][number]+1))
    for i in list_num:
        csselector = site["part_css"][number]%i
        iframe_exist = site["iframe_여부"][number]
        if iframe_exist == 0:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()
        elif iframe_exist == 1:
            driver.switch_to_default_content()
            iframe_num = int(site["iframe"][number])
            iframe = driver.find_elements_by_tag_name("iframe")[iframe_num]
            driver.switch_to_frame(iframe)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()
        part_save2(number)
        data_frame = data_frame.append(save2(number))
        time.sleep(2)        
        driver.get(urls[number])
    os.chdir("C:/Users/park/Desktop/sample")    
    data_frame.to_excel('%s.xlsx'%site['부처'][number], index=False)

In [40]:
def iframe(number):
    global driver,site
    driver.switch_to_default_content()
    iframe_num = int(site["iframe"][number])
    iframe = driver.find_elements_by_tag_name("iframe")[iframe_num]
    driver.switch_to_frame(iframe)

In [355]:
for j in range(7,len(site)):
    A_site2(j)
    time.sleep(2)        

C:\Users\park\Anaconda3\envs\crawler\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use driver.switch_to.default_content instead
  del sys.path[0]
C:\Users\park\Anaconda3\envs\crawler\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use driver.switch_to.frame instead
  app.launch_new_instance()


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [13]:
driver.get(urls[9])

In [15]:
part_save2(9)

('[공지] 2020 KFA 공인구 현황 안내',
 '2020-04-13 10:26:38',
 ['2020 KFA 공인구 현황을 첨부와 같이 안내하오니, 참고하여 주시기 바랍니다.\ufeff 끝. '],
 ['http://img01.kfa.or.kr/bbs/notice/2020%20kfa%20official%20ball%20list.pdf'],
 ['2020 kfa official ball list.pdf'],
 '-')

In [20]:
save2(9)

Failed to create directory!!!!!


,제목,올린날,내용,첨부파일갯수
0,19년 국민참여예산 사업제안 집중 접수 기간 안내 알림,2019.04.02,"[https://youtu.be/duvMlYGs3tQ , “내가 제안하는 사업이 ...",0


In [41]:
A_site2(9)

Failed to create directory!!!!!
Failed to create directory!!!!!


TimeoutException: Message: 


In [70]:
list_num = range(int(site["range_row"][9]),int(site["range_high"][9]+1))
for i in list_num:
    menu = driver.find_element_by_css_selector(site["menu_css"][9])
    menu.find_elements_by_css_selector(site["part_css"][9])[i].click()
    time.sleep(1)
    driver.get(urls[9])
    time.sleep(1)
#hidden_submenu = driver.find_element_by_css_selector("#location_view_915 > td:nth-child(2)")
#actions = ActionChains(driver)
#actions.move_to_element(menu)
#actions.click(hidden_submenu)
#actions.perform()

IndexError: list index out of range

In [75]:
menu.find_elements_by_css_selector(site["part_css"][9])[15]

IndexError: list index out of range

In [71]:
list_num

range(2, 16)

In [18]:
actions.perform()

JavascriptException: Message: javascript error: Failed to execute 'elementsFromPoint' on 'Document': The provided double value is non-finite.
  (Session info: chrome=81.0.4044.122)


In [123]:
def A_site3(number):
    global site,driver
    data_frame = pd.DataFrame(columns = ["제목","올린날","내용","첨부파일갯수"])
    driver.get(urls[number])
    driver.implicitly_wait(2)
    list_num = range(int(site["range_row"][number]),int(site["range_high"][number]+1))
    if number == 4:
        time.sleep(10)
        driver.refresh()
    for i,j in zip(list_num,list_num):
        try:
            csselector = site["part_css"][number]%i
        except:
            pass
        iframe_exist = site["iframe_여부"][number]
        if iframe_exist == 0:
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()
            except:
                menu = driver.find_element_by_css_selector(site["menu_css"][number])
                part = menu.find_elements_by_css_selector(site["part_css1"][number])[j]
                part.click()
        elif iframe_exist == 1:
            driver.switch_to_default_content()
            iframe_num = int(site["iframe"][number])
            iframe = driver.find_elements_by_tag_name("iframe")[iframe_num]
            driver.switch_to_frame(iframe)
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()
            except:
                menu = driver.find_element_by_css_selector(site["menu_css"][number])
                menu.find_elements_by_css_selector(site["part_css"][number])[j].click()     
        part_save2(number)
        data_frame = data_frame.append(save2(number))
        driver.implicitly_wait(2)    
        driver.get(urls[number])
    os.chdir("C:/Users/park/Desktop/sample")    
    data_frame.to_excel('%s.xlsx'%site['부처'][number], index=False)

In [21]:
global driver
driver = webdriver.Chrome("C:/Users/park/Desktop/사이트크롤링/chromedriver.exe")
driver.implicitly_wait(3)

In [124]:
A_site3(5)

C:\Users\park\Anaconda3\envs\crawler\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.default_content instead
  This is separate from the ipykernel package so we can avoid doing imports until


IndexError: list index out of range

In [26]:
for j in range(4,len(site)):
    A_site3(j)
    time.sleep(2)        

100% [..............................................................................] 27136 / 27136

C:\Users\park\Anaconda3\envs\crawler\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.default_content instead
  This is separate from the ipykernel package so we can avoid doing imports until


IndexError: list index out of range

In [114]:
EC.presence_of_element_located(part)

In [111]:
csselector

'#theForm > div.board-main > table > tbody > tr:nth-child(15)'

In [121]:
csselector = site["part_css"][5]%2
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()

In [122]:
menu = driver.find_element_by_css_selector(site["menu_css"][5])
part = menu.find_elements_by_css_selector(site["part_css1"][5])[0]
part.click()

In [120]:
A_site3(5)

Failed to create directory!!!!!


C:\Users\park\Anaconda3\envs\crawler\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.default_content instead
  This is separate from the ipykernel package so we can avoid doing imports until


IndexError: list index out of range